# Building Dataset for UEFA Euro 2020 Fantasy Football

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import json

In [2]:
DATA_DIR = '../data'

## Main Dataset

In [3]:
players = pd.read_excel("{}/raw/fantasy_data.xlsx".format(DATA_DIR), sheet_name=0)
goalie = pd.read_excel("{}/raw/fantasy_data.xlsx".format(DATA_DIR), sheet_name=1)

In [4]:
def euro_fantasy_score(df):
    score = 1
    if df["Game Started"] > 0:
        score += 1
    if df["Min"] >= 60:
        score += 1
    if df["Assists"] > 0:
        score += (df["Assists"] * 3)
    if df["Penalty Kick Miss"] > 0:
        score -= (df["Penalty Kick Miss"] * 2)
    if df["Yellow Cards"] > 0:
        score -= 1
    if df["Red Cards"] > 0:
        score -= 3
    if df["Position"] == "F":
        score += (df["Goals"] * 4)
        score += (df["Shots On Goal"] * 1)
        score += (df["Fouls Drawn"] * 0.5)
    if df["Position"] == "M" or df['Position'] == "M/F":
        score += (df["Goals"] * 5)
        if (df["Accurate Passes"] >= 30):
            score += 3
        if df["Min"] >= 60 and df["Clean Sheet"] > 0:
            score += 1
    if df["Position"] == "D":
        score += (df["Goals"] * 6)
        score -= (df["Fouls Committed"] * 0.5)
        if df["Min"] >= 60 and df["Clean Sheet"] > 0:
            score += 4
        score -= (df['Goals Allowed'] // 2)
    if df["Position"] == "GK":
        score += (df["Goals"] * 6)
        score += (df["Penalty Kick Saved"] * 5)
        if df["Min"] >= 60 and df["Clean Sheet"] > 0:
            score += 4
        score += (df["Saves"] // 3)
        score -= (df['Goals Allowed'] // 2)
    return score

In [5]:
def get_agg_before(df):
    df_new = df.copy()
    merged_df = pd.merge(df, df_new, on=["Player", "TeamName"])
    merged_df = merged_df[merged_df['Date_y'] < merged_df["Date_x"]]
    merged_df = merged_df.groupby(["Player", "TeamName", "Date_x"]).agg(prev_mean_euro_score=("euro_score_y", "mean"),
                                                                        prev_mean_goals=("Goals_y", "mean"),
                                                                        prev_mean_assists=("Assists_y", "mean"),
                                                                        prev_median_min=("Min_y", "median"),
                                                                        prev_starter_rate=("Game Started_y", "mean"))
    merged_df = merged_df.reset_index()
    merged_df.rename(columns={"Date_x": "Date"}, inplace=True)
    return merged_df

In [6]:
players['euro_score'] = players.apply(euro_fantasy_score, axis=1)
agg_players = get_agg_before(players)
temp_player = players[["Player", "Date", "LeagueName", "TeamName", "OpponentName", "Position", "euro_score"]]
temp_player = pd.merge(temp_player, agg_players, on=["Player", "TeamName", "Date"])

In [7]:
goalie["euro_score"] = goalie.apply(euro_fantasy_score, axis=1)
agg_goalie = get_agg_before(goalie)
temp_goalie = goalie[["Player", "Date", "LeagueName", "TeamName", "OpponentName", "Position", "euro_score"]]
temp_goalie = pd.merge(temp_goalie, agg_goalie, on=["Player", "TeamName", "Date"])

In [8]:
df = pd.concat([temp_player, temp_goalie])
df["is_friendlies"] = df["LeagueName"] == "International Friendlies"

In [9]:
df.columns

Index(['Player', 'Date', 'LeagueName', 'TeamName', 'OpponentName', 'Position',
       'euro_score', 'prev_mean_euro_score', 'prev_mean_goals',
       'prev_mean_assists', 'prev_median_min', 'prev_starter_rate',
       'is_friendlies'],
      dtype='object')

In [10]:
df.columns = ["player_name", "date", "league_name", "team_name", "opponent_name", "position", "euro_score", 'prev_mean_euro_score', 'prev_mean_goals', 'prev_mean_assists', 'prev_median_min', 'prev_starter_rate', "is_friendlies"]
df = df.sort_values(by=["player_name","date"])

In [11]:
df.loc[df["position"] == "M/F", "position"] = "M"

## FIFA Rank Dataset

In [12]:
df["date"].min()

Timestamp('2018-09-09 00:00:00')

In [13]:
fifa_rank = pd.read_csv("{}/raw/fifa_ranking-2021-05-27.csv".format(DATA_DIR))

In [14]:
fifa_rank = fifa_rank[fifa_rank["rank_date"] > '2018-09-09']
fifa_rank = fifa_rank[["country_full", "rank", "total_points", "rank_date"]]
fifa_rank = fifa_rank.sort_values(by=["country_full", "rank_date"])

In [15]:
fifa_rank

,country_full,rank,total_points,rank_date
58414,Afghanistan,146,1068,2018-09-20
58626,Afghanistan,145,1068,2018-10-25
58836,Afghanistan,147,1068,2018-11-29
59046,Afghanistan,147,1068,2018-12-20
59256,Afghanistan,147,1066,2019-02-07
...,...,...,...,...
62107,Zimbabwe,108,1181,2020-11-26
62241,Zimbabwe,108,1181,2020-12-10
62455,Zimbabwe,112,1176,2021-02-18
62794,Zimbabwe,107,1175,2021-04-07


In [16]:
df_with_rank = pd.merge(df[["team_name", "date"]], fifa_rank, how="left", left_on="team_name", right_on="country_full")
df_with_rank["rank_date"] = pd.to_datetime(df_with_rank['rank_date'])
df_with_rank["time_diff"] = df_with_rank.apply(lambda x: (x['date']-x['rank_date']).total_seconds(), axis=1)
df_with_rank = df_with_rank[df_with_rank["time_diff"] > 0]
df_with_rank = df_with_rank.sort_values(by=["team_name", "time_diff"], ascending=False)
df_with_rank = df_with_rank.groupby(["team_name", "date"]).agg(team_rank=("rank", "last"),
                                                               team_total_points=("total_points", "last")).reset_index()

In [17]:
df = pd.merge(df, df_with_rank, on=["team_name", "date"], how="left")

In [18]:
df_with_rank = pd.merge(df[["opponent_name", "date"]], fifa_rank, how="left", left_on="opponent_name", right_on="country_full")
df_with_rank["rank_date"] = pd.to_datetime(df_with_rank['rank_date'])
df_with_rank["time_diff"] = df_with_rank.apply(lambda x: (x['date']-x['rank_date']).total_seconds(), axis=1)
df_with_rank = df_with_rank[df_with_rank["time_diff"] > 0]
df_with_rank = df_with_rank.sort_values(by=["opponent_name", "time_diff"], ascending=False)
df_with_rank = df_with_rank.groupby(["opponent_name", "date"]).agg(opponent_rank=("rank", "last"),
                                                                   opponent_total_points=("total_points", "last")).reset_index()

In [19]:
df = pd.merge(df, df_with_rank, on=["opponent_name", "date"], how="left")

In [20]:
df.head()

,player_name,date,league_name,team_name,opponent_name,position,euro_score,prev_mean_euro_score,prev_mean_goals,prev_mean_assists,prev_median_min,prev_starter_rate,is_friendlies,team_rank,team_total_points,opponent_rank,opponent_total_points
0,Emerson,2019-06-11,European Championship Qualifiers,Italy,Bosnia and Herzegovina,D,2.5,10.000000,0.0,1.000000,68.0,1.0,False,17.0,1550.0,35.0,1478.0
1,Emerson,2019-09-05,European Championship Qualifiers,Italy,Armenia,D,6.0,6.250000,0.0,0.500000,79.0,1.0,False,16.0,1569.0,98.0,1230.0
2,Emerson,2019-09-08,European Championship Qualifiers,Italy,Finland,D,2.0,6.166667,0.0,0.666667,90.0,1.0,False,16.0,1569.0,57.0,1394.0
3,Emerson,2019-11-15,European Championship Qualifiers,Italy,Bosnia and Herzegovina,D,7.0,5.125000,0.0,0.500000,79.0,1.0,False,15.0,1593.0,48.0,1435.0
4,Jorginho,2019-03-26,European Championship Qualifiers,Italy,Liechtenstein,M,5.0,7.000000,0.0,0.000000,90.0,1.0,False,18.0,1539.0,181.0,937.0


In [21]:
df["fifa_rank_diff"] = df["team_rank"] - df["opponent_rank"]
df["fifa_points_diff"] = df["team_total_points"] - df["opponent_total_points"]

In [22]:
df.to_csv("{}/interim/all_data.csv".format(DATA_DIR), index=False)